# Data Preprocessing - TCGA SIGvival

---

In [1]:
# Import statements
import pandas as pd
import os
import math

## 1) Load Data

---

### 1.1) Transfer CSV Files to Parquet

Original files were downloaded from the Xena website in TSV and probeMap file formats. Transferred into parquet format for faster file read-in.

In [ ]:
# Read in the CSV and probeMap into a DataFrame
df_transfer = pd.read_csv('./data/GDC-PANCAN.htseq_fpkm-uq.tsv', sep='\t')
df_survival_transfer = pd.read_csv('./data/GDC-PANCAN.survival.tsv', sep='\t')
df_basic_pheno_transfer = pd.read_csv('./data/GDC-PANCAN.basic_phenotype.tsv', sep='\t')
df_probeMap_transfer = pd.read_csv('./data/gencode.v22.annotation.gene.probeMap', sep='\t')

In [ ]:
# Transform into parquet files
df_transfer.to_parquet('./data/GDC-PANCAN.htseq_fpkm-uq.parquet', compression=None)
df_survival_transfer.to_parquet('./data/GDC-PANCAN.survival.parquet', compression=None)
df_basic_pheno_transfer.to_parquet('./data/GDC-PANCAN.basic_phenotype.parquet', compression=None)
df_probeMap_transfer.to_parquet('./data/gencode.v22.annotation.gene.parquet', compression=None)

## 2) Format Data

---

### 2.1) Map ID/Genes and Downcast Data Types

Downcast from float64 to float16 for overall file size reduction.

In [2]:
# Read in the RNA matrix
df = pd.read_parquet('./data/GDC-PANCAN.htseq_fpkm-uq.parquet')
# Read in ID/Gene Mapping file
mapping_df = pd.read_parquet('./data/gencode.v22.annotation.gene.parquet')
display(df.head())
display(mapping_df.head())

,xena_sample,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,TCGA-OR-A5JR-01A,TCGA-OR-A5KU-01A,TCGA-OR-A5LS-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5JQ-01A,...,TARGET-50-PAJMKJ-01A,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A
0,ENSG00000242268.2,0.000000,0.000000,0.000000,0.000000,9.486642,0.000000,0.000000,0.000000,0.000000,...,11.700035,10.041859,13.398458,0.000000,10.617230,11.933609,14.140998,11.659218,10.662028,12.878131
1,ENSG00000270112.3,10.689655,14.408626,14.022621,11.291444,10.221394,12.423503,12.830424,12.758888,11.547426,...,9.267574,12.513257,10.501003,10.452072,10.625798,8.310776,7.131909,7.678919,10.134942,11.116645
2,ENSG00000167578.15,18.536987,18.684183,17.334107,19.713465,16.761630,17.762472,18.114361,19.068519,17.474470,...,15.541309,16.684341,15.905948,16.991286,15.066989,13.953978,15.969451,14.607776,14.387707,15.886538
3,ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000078237.5,17.847476,18.227483,17.287893,16.722624,17.157762,17.001996,18.648729,18.076084,15.817248,...,15.377730,16.438256,16.733394,16.149538,16.277784,15.673957,16.305087,15.916629,15.850915,16.188748


,id,gene,chrom,chromStart,chromEnd,strand
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,+
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,-
2,ENSG00000278267.1,MIR6859-3,chr1,17369,17436,-
3,ENSG00000243485.3,RP11-34P13.3,chr1,29554,31109,+
4,ENSG00000274890.1,MIR1302-9,chr1,30366,30503,+


In [3]:
# Rename the mapping file to have the same id names as the RNA matrix file 
mapping_df.rename(columns={'id': 'xena_sample'}, inplace=True)

# Merge the two DataFrames on xena_sample
merged_df = pd.merge(mapping_df, df, on='xena_sample', how='outer', indicator=True)

# Check matching status - filter rows that do not have 'both' in the '_merge' column
non_matching_rows = merged_df[merged_df['_merge'] != 'both']

In [4]:
# Print the non-matching rows
display(non_matching_rows) # None!

,xena_sample,gene,chrom,chromStart,chromEnd,strand,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,...,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A,_merge


In [5]:
# Print the updated DataFrame
display(merged_df.head())

,xena_sample,gene,chrom,chromStart,chromEnd,strand,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,...,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A,_merge
0,ENSG00000000003.13,TSPAN6,chrX,100627109,100639991,-,17.544533,17.623128,17.747056,18.246428,...,19.491339,20.104873,19.215693,19.308517,19.586799,19.567246,18.448389,19.536579,19.907498,both
1,ENSG00000000005.5,TNMD,chrX,100584802,100599885,+,10.592418,11.728494,12.427999,0.000000,...,14.251711,10.770522,13.559971,9.758736,16.225119,11.877669,12.018519,16.369945,11.156833,both
2,ENSG00000000419.11,DPM1,chr20,50934867,50958555,-,20.046039,20.569446,20.392837,18.801009,...,19.230607,19.579990,19.374570,19.258578,19.058986,19.981892,19.085757,19.326476,19.838199,both
3,ENSG00000000457.12,SCYL3,chr1,169849631,169894267,-,15.323361,15.228320,15.075964,15.563598,...,15.451625,15.225552,15.411968,16.505343,15.762901,15.481780,17.207028,16.205465,15.483468,both
4,ENSG00000000460.15,C1orf112,chr1,169662007,169854080,+,14.066195,14.361755,14.457514,13.769704,...,12.789757,14.406227,14.040450,15.908260,15.992289,15.800770,16.772996,16.536652,15.214130,both


In [6]:
# Ensure the expression dataframe is in the format: indexed on gene names column labels as sample ids
merged_trimmed_df = merged_df.copy()

# Drop unnecessary columns
merged_trimmed_df.drop(columns=['xena_sample', 'chrom', 'chromStart', 'chromEnd', 'strand', '_merge'], axis=1, inplace=True)

# Set the index as gene names
merged_trimmed_df.set_index('gene', inplace=True)
display(merged_trimmed_df.head())
print(merged_trimmed_df.info())

,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,TCGA-OR-A5JR-01A,TCGA-OR-A5KU-01A,TCGA-OR-A5LS-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5JQ-01A,TCGA-OR-A5JS-01A,...,TARGET-50-PAJMKJ-01A,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,17.544533,17.623128,17.747056,18.246428,18.257023,18.353113,18.292785,17.084432,18.562685,18.237882,...,19.286487,19.491339,20.104873,19.215693,19.308517,19.586799,19.567246,18.448389,19.536579,19.907498
TNMD,10.592418,11.728494,12.427999,0.000000,8.308020,9.006822,13.378305,11.823974,9.672630,9.578988,...,9.520397,14.251711,10.770522,13.559971,9.758736,16.225119,11.877669,12.018519,16.369945,11.156833
DPM1,20.046039,20.569446,20.392837,18.801009,20.042714,20.133037,19.859895,19.585215,19.886858,20.067324,...,19.861846,19.230607,19.579990,19.374570,19.258578,19.058986,19.981892,19.085757,19.326476,19.838199
SCYL3,15.323361,15.228320,15.075964,15.563598,15.260088,14.850601,16.187010,15.453456,15.045246,15.309825,...,16.511700,15.451625,15.225552,15.411968,16.505343,15.762901,15.481780,17.207028,16.205465,15.483468
C1orf112,14.066195,14.361755,14.457514,13.769704,14.161787,13.557133,13.918952,15.328623,13.781018,14.321432,...,16.570038,12.789757,14.406227,14.040450,15.908260,15.992289,15.800770,16.772996,16.536652,15.214130


<class 'pandas.core.frame.DataFrame'>
Index: 60483 entries, TSPAN6 to RP13-465B17.4
Columns: 11768 entries, TCGA-OR-A5JP-01A to TARGET-50-PALFRD-01A
dtypes: float64(11768)
memory usage: 5.3+ GB
None


In [7]:
# Downcast data type from float64 to float16
merged_trimmed_df = merged_trimmed_df.astype('float16')

In [ ]:
# Save to file
merged_trimmed_df.to_parquet('./data/GDC-PANCAN.htseq_fpkm-uq_float16.parquet', compression=None)

### 2.2) Split RNA Parquet File into Cancer Types

For GitHub Data Storage. The file was originally too large to push to GitHub.

In [8]:
# Read in the phenotype and survival DataFrames
phenotype_df = pd.read_parquet('./data/GDC-PANCAN.basic_phenotype.parquet')
survival_df = pd.read_parquet('./data/GDC-PANCAN.survival.parquet')

In [9]:
# Display the RNA and phenotype DataFrames
display(merged_trimmed_df.shape)
display(phenotype_df.shape)
display(survival_df.shape)

(60483, 11768)

(19188, 7)

(18492, 4)

In [10]:
# Locate all cancer types
cancer_type_list = list(phenotype_df['project_id'].unique())

#### 2.2.1) Align RNA / OS / Phenotype samples

Note: The RNA matrix column headers are TCGA barcodes:
* In the format of: `project-tissuesourcesite-participant-sample|vial-portion|analyte-plate-center`
* https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/
* https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/tissue-source-site-codes

In [11]:
# Subset the survival_df and phenotype_df to only include samples present in RNA data
# Create a list of all samples in each dataframe
samples_rna = list(merged_trimmed_df.columns)
samples_pheno = list(phenotype_df['sample'].values)
samples_survival = list(survival_df['sample'].values)

# Find all common samples in all three lists
common_samples = list(set(samples_rna) & set(samples_pheno) & set(samples_survival))

# Subset and reorder all three datasets by common_samples
merged_trimmed_filtered_df = merged_trimmed_df[common_samples] # Filter merged_df by columns in common_samples
phenotype_filtered_df = phenotype_df[phenotype_df['sample'].isin(common_samples)]
survival_filtered_df = survival_df[survival_df['sample'].isin(common_samples)]

In [12]:
# Reorder phenotype and survival dataframes to match the columns of the rna matrix
column_order = list(merged_trimmed_filtered_df.columns)
phenotype_filtered_ordered_df = phenotype_filtered_df.set_index('sample').loc[column_order].reset_index()
survival_filtered_ordered_df = survival_filtered_df.set_index('sample').loc[column_order].reset_index()

In [13]:
# Save each cancer type to their own individual files
# Loop through each cancer type
for cancer_type in cancer_type_list:
    # Locate all samples in phenotype_df that match the cancer type
    samples = phenotype_df[phenotype_df['project_id'] == cancer_type]['sample']

    # Only select samples that are present in merged_trimmed_df
    valid_samples = samples[samples.isin(merged_trimmed_df.columns)]
    
    # Subset merged_trimmed_df to just the samples that are in the cancer type
    samples_df = merged_trimmed_df[valid_samples]
    # samples_df = phenotype_df[phenotype_df['sample'].isin(samples)] # What I had before
    
    # Write out to file
    samples_df.to_parquet(f'./data/GDC-PANCAN.htseq_fpkm-uq_{cancer_type}.parquet', compression=None)

In [14]:
# TEST that the files wrote out correctly
tcga_brca_df = pd.read_parquet('./data/GDC-PANCAN.htseq_fpkm-uq_TCGA-BRCA.parquet')
display(tcga_brca_df)

,TCGA-AR-A24Z-01A,TCGA-D8-A143-01A,TCGA-B6-A0IC-01A,TCGA-A8-A0A4-01A,TCGA-B6-A0WZ-01A,TCGA-A8-A08A-01A,TCGA-AR-A1AN-01A,TCGA-AO-A0J4-01A,TCGA-A2-A0D3-01A,TCGA-BH-A0H3-01A,...,TCGA-AR-A24S-01A,TCGA-A1-A0SH-01A,TCGA-E9-A22A-01A,TCGA-GI-A2C8-11A,TCGA-AO-A0J8-01A,TCGA-BH-A1FE-01A,TCGA-C8-A1HN-01A,TCGA-D8-A1JB-01A,TCGA-A7-A0DA-01A,TCGA-E2-A15C-01A
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,18.000000,19.078125,15.914062,18.046875,17.390625,18.609375,16.687500,17.531250,19.125000,18.984375,...,17.796875,18.28125,16.781250,15.734375,17.765625,17.968750,17.703125,18.250000,17.687500,18.687500
TNMD,11.585938,7.980469,7.304688,9.062500,9.937500,11.054688,13.156250,8.921875,11.195312,11.734375,...,14.406250,9.50000,11.562500,12.507812,12.382812,9.976562,8.414062,10.710938,10.570312,10.554688
DPM1,19.562500,19.750000,18.828125,18.890625,18.812500,18.531250,18.750000,19.328125,19.171875,19.062500,...,19.343750,18.90625,19.187500,19.625000,18.828125,19.609375,20.703125,20.031250,19.125000,19.218750
SCYL3,15.710938,16.734375,16.687500,16.546875,16.671875,17.031250,16.296875,15.750000,17.937500,17.062500,...,16.234375,16.00000,16.437500,15.960938,17.328125,16.578125,17.109375,16.937500,16.609375,16.765625
C1orf112,14.687500,17.296875,15.187500,15.304688,14.703125,15.132812,15.046875,15.164062,16.046875,15.460938,...,15.890625,14.50000,15.179688,13.078125,15.804688,15.710938,15.562500,14.796875,15.632812,15.117188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Metazoa_SRP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AJ271736.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MIR6089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
# Output phenotype and survival data to file
phenotype_filtered_ordered_df.to_parquet('./data/GDC-PANCAN.basic_phenotype_processed.parquet', compression=None)
survival_filtered_ordered_df.to_parquet('./data/GDC-PANCAN.survival_processed.parquet', compression=None)

### 2.3) Split TCGA-BRCA RNA Cancer Type Parquet File

The file size for `GDC-PANCAN.htseq_fpkm-uq_TCGA-BRCA.parquet` is over the GitHub push limit of 100MiB per file.

In [16]:
half = math.ceil(tcga_brca_df.shape[1] / 2) # 1217 columns / 2 = 608.5 ceiling is 609
tcga_brca_1_df = tcga_brca_df.iloc[:, :half]
tcga_brca_2_df = tcga_brca_df.iloc[:, half:]

# Confirm TCGA-BRCA DataFrame is split correctly
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(tcga_brca_1_df.tail())
display(tcga_brca_2_df.head())
# print(tcga_brca_df.columns.tolist()) # Used to confirm

,TCGA-AR-A24Z-01A,TCGA-D8-A143-01A,TCGA-B6-A0IC-01A,TCGA-A8-A0A4-01A,TCGA-B6-A0WZ-01A,TCGA-A8-A08A-01A,TCGA-AR-A1AN-01A,TCGA-AO-A0J4-01A,TCGA-A2-A0D3-01A,TCGA-BH-A0H3-01A,TCGA-LL-A5YN-01A,TCGA-AR-A1AP-01A,TCGA-E2-A1IO-01A,TCGA-EW-A1P8-01A,TCGA-5L-AAT1-01A,TCGA-A8-A06X-01A,TCGA-B6-A3ZX-01A,TCGA-E2-A10E-01A,TCGA-AN-A0XL-01A,TCGA-A2-A04W-01A,TCGA-BH-A0HL-01A,TCGA-E9-A1N4-11A,TCGA-A8-A093-01A,TCGA-A8-A090-01A,TCGA-D8-A1X8-01A,TCGA-B6-A0IP-01A,TCGA-A8-A09T-01A,TCGA-BH-A0B4-01A,TCGA-A7-A0CJ-01A,TCGA-D8-A27L-01A,TCGA-A7-A6VY-01A,TCGA-B6-A0IE-01A,TCGA-E9-A24A-01A,TCGA-AR-A1AS-01A,TCGA-C8-A12K-01A,TCGA-AC-A2QI-01A,TCGA-AC-A6IX-01A,TCGA-AN-A0FK-01A,TCGA-C8-A27B-01A,TCGA-E2-A15S-01A,TCGA-OL-A5RX-01A,TCGA-BH-A0H5-01A,TCGA-A2-A1G6-01A,TCGA-A7-A4SD-01A,TCGA-BH-A0BM-11A,TCGA-BH-A0HW-01A,TCGA-BH-A8G0-01A,TCGA-A8-A06T-01A,TCGA-LD-A7W6-01A,TCGA-AN-A0XP-01A,TCGA-AN-A0XN-01A,TCGA-EW-A1P5-01A,TCGA-A8-A0A1-01A,TCGA-BH-A0BC-11A,TCGA-A7-A5ZV-01A,TCGA-BH-A0B7-01A,TCGA-E2-A14T-01A,TCGA-E2-A1II-01A,TCGA-AQ-A54O-01A,TCGA-AR-A2LL-01A,TCGA-E9-A54Y-01A,TCGA-C8-A12V-01A,TCGA-AR-A24R-01A,TCGA-BH-A0EB-01A,TCGA-LL-A50Y-01A,TCGA-BH-A0GZ-01A,TCGA-BH-A0AZ-01A,TCGA-E2-A15A-01A,TCGA-D8-A1JT-01A,TCGA-A8-A09G-01A,TCGA-AO-A0JL-01A,TCGA-D8-A1JI-01A,TCGA-A8-A0A6-01A,TCGA-D8-A1Y3-01A,TCGA-A8-A081-01A,TCGA-E2-A573-01A,TCGA-A8-A07F-01A,TCGA-AR-A2LR-01A,TCGA-AR-A2LE-01A,TCGA-A2-A0YE-01A,TCGA-OL-A5RU-01A,TCGA-AR-A1AV-01A,TCGA-A2-A04Q-01A,TCGA-C8-A1HM-01A,TCGA-B6-A0X1-01A,TCGA-AR-A0TX-01A,TCGA-D8-A27G-01A,TCGA-D8-A27W-01A,TCGA-BH-A0GY-01A,TCGA-AQ-A7U7-01A,TCGA-OL-A5RV-01A,TCGA-BH-A0C3-11A,TCGA-BH-A0HN-01A,TCGA-Z7-A8R5-01A,TCGA-E2-A158-01A,TCGA-AR-A24O-01A,TCGA-A2-A0EX-01A,TCGA-A8-A06Z-01A,TCGA-C8-A1HJ-01A,TCGA-A7-A26G-01A,TCGA-E2-A15P-01A,TCGA-LL-A5YM-01A,TCGA-A8-A09K-01A,TCGA-A7-A0CD-01A,TCGA-D8-A1XZ-01A,TCGA-AR-A24K-01A,TCGA-C8-A8HQ-01A,TCGA-C8-A1HL-01A,TCGA-A7-A13G-11A,TCGA-B6-A0RO-01A,TCGA-B6-A0IG-01A,TCGA-AR-A1AQ-01A,TCGA-A2-A4S3-01A,TCGA-BH-A204-01A,TCGA-A2-A1G0-01A,TCGA-AN-A0FY-01A,TCGA-BH-A0DS-01A,TCGA-AO-A0J7-01A,TCGA-D8-A140-01A,TCGA-AC-A8OP-01A,TCGA-A8-A06O-01A,TCGA-LL-A740-01A,TCGA-E2-A1B6-01A,TCGA-S3-A6ZF-01A,TCGA-AC-A6IV-01A,TCGA-EW-A1PH-01A,TCGA-AO-A03V-01A,TCGA-E9-A244-01A,TCGA-A8-A07P-01A,TCGA-A7-A26F-01B,TCGA-A7-A0D9-11A,TCGA-E9-A1N6-11A,TCGA-AC-A7VB-01A,TCGA-A7-A3J1-01A,TCGA-E9-A1ND-11A,TCGA-D8-A141-01A,TCGA-A7-A26H-01A,TCGA-BH-A1FN-11A,TCGA-B6-A0RL-01A,TCGA-E2-A154-01A,TCGA-A2-A0T2-01A,TCGA-A2-A0D1-01A,TCGA-AC-A2FO-01A,TCGA-UU-A93S-01A,TCGA-S3-AA17-01A,TCGA-OL-A66P-01A,TCGA-LL-A5YO-01A,TCGA-BH-A203-11A,TCGA-A2-A0CT-01A,TCGA-A8-A06U-01A,TCGA-BH-A28Q-01A,TCGA-A2-A0T1-01A,TCGA-BH-A18V-06A,TCGA-A8-A08B-01A,TCGA-GM-A2DN-01A,TCGA-BH-A0DP-01A,TCGA-A2-A0CR-01A,TCGA-AC-A8OS-01A,TCGA-B6-A0WW-01A,TCGA-3C-AALK-01A,TCGA-BH-A0DE-01A,TCGA-AO-A0JA-01A,TCGA-A7-A0CH-11A,TCGA-AO-A1KS-01A,TCGA-AO-A0J3-01A,TCGA-AR-A1AM-01A,TCGA-E2-A1BC-01A,TCGA-E9-A1RH-01A,TCGA-C8-A278-01A,TCGA-AC-A3QQ-01B,TCGA-EW-A2FW-01A,TCGA-AC-A23G-01A,TCGA-BH-A0B6-01A,TCGA-E2-A1LE-01A,TCGA-PE-A5DD-01A,TCGA-A2-A04Y-01A,TCGA-BH-A1F8-01A,TCGA-BH-A18U-01A,TCGA-A7-A4SF-01A,TCGA-D8-A1XQ-01A,TCGA-A2-A0SY-01A,TCGA-AN-A0AK-01A,TCGA-C8-A12N-01A,TCGA-AO-A12C-01A,TCGA-D8-A27I-01A,TCGA-D8-A1Y2-01A,TCGA-E2-A1IH-01A,TCGA-A2-A1FZ-01A,TCGA-A2-A0SV-01A,TCGA-D8-A27F-01A,TCGA-AC-A23C-01A,TCGA-AR-A1AI-01A,TCGA-E2-A1LS-01A,TCGA-AN-A049-01A,TCGA-E9-A3HO-01A,TCGA-A2-A0CY-01A,TCGA-A8-A084-01A,TCGA-E2-A15K-11A,TCGA-A2-A0CV-01A,TCGA-OL-A5DA-01A,TCGA-A2-A04N-01A,TCGA-B6-A0RI-01A,TCGA-A2-A4S2-01A,TCGA-AO-A129-01A,TCGA-AR-A1AW-01A,TCGA-AC-A2FG-01A,TCGA-AC-A3TN-01A,TCGA-AO-A03U-01B,TCGA-A2-A0EM-01A,TCGA-D8-A1JL-01A,TCGA-A8-A09E-01A,TCGA-AO-A0JD-01A,TCGA-A2-A0EU-01A,TCGA-E2-A15E-06A,TCGA-BH-A0AZ-11A,TCGA-BH-A1EO-11A,TCGA-GM-A2DO-01A,TCGA-B6-A401-01A,TCGA-A1-A0SE-01A,TCGA-E2-A1LK-01A,TCGA-AO-A1KT-01A,TCGA-LL-A73Y-01A,TCGA-BH-A0B9-01A,TCGA-BH-A0W7-01A,TCGA-AN-A0FD-01A,TCGA-E2-A155-01A,TCGA-A8-A08I-01A,TCGA-LL-A6FR-01A,TCGA-AC-A3HN-01A,TCGA-B6-A1KN-01A,TCGA-B6-A0RN-01A,TCGA-A2-A25D-01A,TCGA-AR-A2LM-01A,TCGA-BH-A0BD-01A,TCGA-A8-A094-01A,TCGA

,TCGA-S3-AA14-01A,TCGA-AR-A24V-01A,TCGA-C8-A26W-01A,TCGA-A8-A07R-01A,TCGA-AR-A1AJ-01A,TCGA-A7-A3J0-01A,TCGA-AN-A046-01A,TCGA-EW-A6SC-01A,TCGA-E2-A10C-01A,TCGA-D8-A27E-01A,TCGA-A2-A0CX-01A,TCGA-D8-A73W-01A,TCGA-A8-A07B-01A,TCGA-AC-A23E-01A,TCGA-A8-A09D-01A,TCGA-BH-A1FN-01A,TCGA-E2-A1LH-01A,TCGA-A8-A09Q-01A,TCGA-E9-A1N6-01A,TCGA-E9-A1RI-01A,TCGA-A7-A4SC-01A,TCGA-A2-A0EO-01A,TCGA-AR-A24M-01A,TCGA-A2-A3Y0-01A,TCGA-EW-A1IW-01A,TCGA-E2-A1IF-01A,TCGA-AR-A1AX-01A,TCGA-BH-A42V-01A,TCGA-A8-A08J-01A,TCGA-BH-A0DQ-11A,TCGA-EW-A1P6-01A,TCGA-AC-A2QJ-01A,TCGA-A8-A097-01A,TCGA-A8-A091-01A,TCGA-AR-A252-01A,TCGA-E9-A1NE-01A,TCGA-C8-A1HI-01A,TCGA-A1-A0SD-01A,TCGA-BH-A0DK-11A,TCGA-JL-A3YX-01A,TCGA-AR-A1AR-01A,TCGA-AR-A24X-01A,TCGA-5T-A9QA-01A,TCGA-A7-A6VX-01A,TCGA-E9-A1N9-01A,TCGA-A2-A1G1-01A,TCGA-BH-A1FR-11B,TCGA-E2-A1IK-01A,TCGA-E2-A15R-01A,TCGA-E2-A108-01A,TCGA-AO-A0JB-01A,TCGA-AO-A03T-01A,TCGA-A7-A4SE-01A,TCGA-AO-A126-01A,TCGA-PE-A5DE-01A,TCGA-BH-A0H7-01A,TCGA-D8-A1XM-01A,TCGA-BH-A1FH-11B,TCGA-D8-A1J9-01A,TCGA-D8-A1JD-01A,TCGA-D8-A1JK-01A,TCGA-A8-A09X-01A,TCGA-A1-A0SF-01A,TCGA-A2-A0YD-01A,TCGA-GM-A2DD-01A,TCGA-D8-A27N-01A,TCGA-E2-A15E-01A,TCGA-C8-A138-01A,TCGA-AO-A12G-01A,TCGA-A2-A0YT-01A,TCGA-PE-A5DC-01A,TCGA-E9-A1NC-01A,TCGA-AR-A5QQ-01A,TCGA-A2-A0CZ-01A,TCGA-D8-A1XS-01A,TCGA-AC-A2QH-01B,TCGA-A7-A0DB-11A,TCGA-A7-A0DB-01A,TCGA-E2-A1B1-01A,TCGA-E9-A22E-01A,TCGA-D8-A3Z5-01A,TCGA-BH-A1FB-01A,TCGA-B6-A0WX-01A,TCGA-E9-A1N9-11A,TCGA-E2-A15H-01A,TCGA-BH-A0HO-01A,TCGA-AN-A0XR-01A,TCGA-BH-A1EN-01A,TCGA-BH-A18Q-01A,TCGA-AQ-A04L-01B,TCGA-5L-AAT0-01A,TCGA-A2-A0EY-01A,TCGA-C8-A26V-01A,TCGA-E9-A22B-01A,TCGA-BH-A0AU-11A,TCGA-E9-A1NA-01A,TCGA-A7-A13D-01B,TCGA-A2-A3KC-01A,TCGA-AN-A0XT-01A,TCGA-E2-A159-01A,TCGA-D8-A73U-01A,TCGA-OL-A5D7-01A,TCGA-A7-A6VW-01A,TCGA-E9-A1NF-01A,TCGA-EW-A1P7-01A,TCGA-E2-A14N-01A,TCGA-B6-A0IB-01A,TCGA-E2-A1IL-01A,TCGA-AC-A3EH-01A,TCGA-AR-A0TS-01A,TCGA-D8-A1Y1-01A,TCGA-BH-A1F2-01A,TCGA-OL-A6VO-01A,TCGA-AR-A0TR-01A,TCGA-AN-A0XO-01A,TCGA-E9-A1R0-01A,TCGA-AR-A24W-01A,TCGA-EW-A3U0-01A,TCGA-A8-A076-01A,TCGA-D8-A1XK-01A,TCGA-E9-A6HE-01A,TCGA-E2-A1L7-01A,TCGA-E9-A1RB-11A,TCGA-E2-A1BD-01A,TCGA-B6-A0RM-01A,TCGA-XX-A89A-01A,TCGA-AO-A0JI-01A,TCGA-AN-A0AM-01A,TCGA-BH-A1FM-11B,TCGA-A2-A0CU-01A,TCGA-B6-A0WV-01A,TCGA-4H-AAAK-01A,TCGA-B6-A0RU-01A,TCGA-BH-A0DO-11A,TCGA-BH-A18K-11A,TCGA-AN-A0FX-01A,TCGA-AR-A0U4-01A,TCGA-AR-A250-01A,TCGA-BH-A0DT-11A,TCGA-A8-A07E-01A,TCGA-AC-A3TM-01A,TCGA-AR-A1AU-01A,TCGA-BH-A1F6-11B,TCGA-C8-A273-01A,TCGA-A7-A0CE-11A,TCGA-A2-A0YI-01A,TCGA-E2-A15A-06A,TCGA-AR-A0TQ-01A,TCGA-AO-A124-01A,TCGA-AC-A2FB-11A,TCGA-AO-A1KQ-01A,TCGA-BH-A1F0-01A,TCGA-D8-A1XL-01A,TCGA-EW-A1PE-01A,TCGA-B6-A0IA-01A,TCGA-A1-A0SQ-01A,TCGA-A7-A26J-01A,TCGA-A7-A26J-01B,TCGA-E9-A1N3-01A,TCGA-E2-A1IG-11A,TCGA-BH-A1EW-11B,TCGA-AC-A6IX-06A,TCGA-BH-A0BR-01A,TCGA-E9-A22D-01A,TCGA-BH-A1FR-01A,TCGA-BH-A0RX-01A,TCGA-BH-A5IZ-01A,TCGA-C8-A12X-01A,TCGA-E2-A15M-11A,TCGA-BH-A0W3-01A,TCGA-E2-A156-01A,TCGA-B6-A0IJ-01A,TCGA-C8-A12U-01A,TCGA-B6-A0I8-01A,TCGA-BH-A0BW-11A,TCGA-LL-A5YP-01A,TCGA-BH-A209-01A,TCGA-AN-A0FT-01A,TCGA-OL-A66L-01A,TCGA-E2-A1LB-01A,TCGA-A8-A08C-01A,TCGA-AO-A1KR-01A,TCGA-BH-A0BA-01A,TCGA-A2-A0YM-01A,TCGA-AO-A12H-01A,TCGA-A8-A07U-01A,TCGA-EW-A2FV-01A,TCGA-E2-A14V-01A,TCGA-A2-A04T-01A,TCGA-EW-A424-01A,TCGA-E2-A1B4-01A,TCGA-A8-A08T-01A,TCGA-BH-A18P-11A,TCGA-A8-A092-01A,TCGA-AC-A8OQ-01A,TCGA-A7-A6VV-01A,TCGA-BH-A0DL-11A,TCGA-B6-A0IK-01A,TCGA-LL-A5YL-01A,TCGA-BH-A0BQ-01A,TCGA-D8-A4Z1-01A,TCGA-A1-A0SG-01A,TCGA-A2-A0CQ-01A,TCGA-BH-A1FE-11B,TCGA-BH-A0DH-11A,TCGA-E9-A1RC-01A,TCGA-A2-A25E-01A,TCGA-AR-A5QM-01A,TCGA-B6-A1KF-01A,TCGA-BH-A1FU-11A,TCGA-A2-A3XW-01A,TCGA-D8-A1JM-01A,TCGA-E2-A15O-01A,TCGA-BH-A0C0-11A,TCGA-BH-A18K-01A,TCGA-AN-A0AT-01A,TCGA-A8-A0A9-01A,TCGA-BH-A18V-01A,TCGA-LL-A9Q3-01A,TCGA-C8-A27A-01A,TCGA-A8-A085-01A,TCGA-Z7-A8R6-01A,TCGA-BH-A0WA-01A,TCGA-E2-A1IN-01A,TCGA-GI-A2C9-11A,TCGA-A2-A0YH-01A,TCGA-BH-A2L8-01A,TCGA-AO-A12F-01A,TCGA-PL-A8LY-01A,TCGA-AC-A23H-11A,TCGA-AN-A041-01A,TCGA-A2-A0YG-01A,TCGA-E9-A1R2-01A,TCGA-A2-A3XY-01A,TCGA-E2-A105-01A,TCGA

In [17]:
# Write out to file
tcga_brca_1_df.to_parquet('./data/GDC-PANCAN.htseq_fpkm-uq_TCGA-BRCA_1.parquet', compression=None)
tcga_brca_2_df.to_parquet('./data/GDC-PANCAN.htseq_fpkm-uq_TCGA-BRCA_2.parquet', compression=None)

### 2.4) Read and Concatenate RNA Cancer Type Parquet Files

In [19]:
# Identify folder where the files are stored
data_folder = './data/'

# TEST - Example of a user-selected list of cancer types to match in the file names
cancer_type_list = ['TCGA-BRCA', 'TCGA-CHOL']

# Define an empty list to hold all the loaded DataFrames
df_list = []

# Loop through each cancer type
for cancer_type in cancer_type_list:
    # Construct the file name pattern to look for
    file_name = f'GDC-PANCAN.htseq_fpkm-uq_{cancer_type}.parquet'
    
    # Check if the file exists in the data folder
    file_path = os.path.join(data_folder, file_name)
    if os.path.exists(file_path):
        # Read the parquet file
        df = pd.read_parquet(file_path)
        df_list.append(df)
        print(f"Loaded data for {cancer_type}, shape: {df.shape}")
    else:
        print(f"File for {cancer_type} not found.")

Loaded data for TCGA-BRCA, shape: (60483, 1217)
Loaded data for TCGA-CHOL, shape: (60483, 45)


In [20]:
# Display the value counts for each cancer type
print(phenotype_filtered_df['project_id'].value_counts())

# Output confirms TCGA-BRCA and TCGA-CHOL are 1217 and 45 columns, respectively

project_id
TCGA-BRCA        1217
TCGA-KIRC         607
TCGA-LUAD         585
TCGA-UCEC         583
TCGA-THCA         568
TCGA-PRAD         551
TCGA-LUSC         550
TCGA-HNSC         546
TCGA-LGG          529
TCGA-COAD         512
TCGA-SKCM         472
TCGA-BLCA         430
TCGA-LIHC         424
TCGA-STAD         407
TCGA-OV           379
TCGA-KIRP         321
TCGA-CESC         309
TCGA-SARC         265
TARGET-AML        187
TCGA-PCPG         186
TCGA-PAAD         182
TCGA-READ         177
TCGA-GBM          173
TCGA-ESCA         173
TCGA-TGCT         156
TCGA-LAML         151
TARGET-NBL        150
TARGET-WT         132
TCGA-THYM         121
TARGET-ALL-P3     108
TCGA-KICH          89
TCGA-MESO          86
TARGET-OS          85
TCGA-UVM           80
TCGA-ACC           79
TCGA-UCS           56
TCGA-DLBC          48
TCGA-CHOL          45
TARGET-CCSK        13
TARGET-RT           5
Name: count, dtype: int64


In [24]:
# Concatenate the selected cancer type DataFrames together
df = pd.concat(df_list, axis=1)
display(df.head())

,TCGA-AR-A24Z-01A,TCGA-D8-A143-01A,TCGA-B6-A0IC-01A,TCGA-A8-A0A4-01A,TCGA-B6-A0WZ-01A,TCGA-A8-A08A-01A,TCGA-AR-A1AN-01A,TCGA-AO-A0J4-01A,TCGA-A2-A0D3-01A,TCGA-BH-A0H3-01A,...,TCGA-ZH-A8Y4-01A,TCGA-4G-AAZT-01A,TCGA-W5-AA2W-01A,TCGA-ZH-A8Y1-01A,TCGA-W5-AA2T-01A,TCGA-ZH-A8Y2-01A,TCGA-W5-AA39-01A,TCGA-3X-AAVA-01A,TCGA-W5-AA2U-01A,TCGA-3X-AAVE-01A
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,18.000000,19.078125,15.914062,18.046875,17.390625,18.609375,16.687500,17.531250,19.125000,18.984375,...,18.703125,20.234375,17.296875,18.015625,20.156250,19.687500,18.390625,19.156250,19.578125,19.140625
TNMD,11.585938,7.980469,7.304688,9.062500,9.937500,11.054688,13.156250,8.921875,11.195312,11.734375,...,0.000000,0.000000,8.484375,0.000000,8.312500,0.000000,9.460938,8.601562,8.210938,0.000000
DPM1,19.562500,19.750000,18.828125,18.890625,18.812500,18.531250,18.750000,19.328125,19.171875,19.062500,...,18.890625,19.390625,19.421875,18.640625,19.281250,19.187500,19.140625,19.296875,18.843750,19.375000
SCYL3,15.710938,16.734375,16.687500,16.546875,16.671875,17.031250,16.296875,15.750000,17.937500,17.062500,...,16.000000,15.406250,17.000000,16.984375,16.890625,16.171875,16.671875,15.953125,16.328125,16.421875
C1orf112,14.687500,17.296875,15.187500,15.304688,14.703125,15.132812,15.046875,15.164062,16.046875,15.460938,...,14.742188,13.945312,15.796875,15.445312,15.609375,15.289062,15.250000,14.609375,14.414062,15.039062


In [25]:
# Confirm the same shape of the new DataFrame
print("Shape ==>", df.shape)
print(df.info())

Shape ==> (60483, 1262)
<class 'pandas.core.frame.DataFrame'>
Index: 60483 entries, TSPAN6 to RP13-465B17.4
Columns: 1262 entries, TCGA-AR-A24Z-01A to TCGA-3X-AAVE-01A
dtypes: float16(1262)
memory usage: 146.0+ MB
None
